# Create a retreat mask from basins and Calfin

In [1]:
import geopandas as gp
gp.options.io_engine = "pyogrio"
import pandas as pd
from tqdm.auto import tqdm
import xarray as xr
from geocube.api.core import make_geocube
import numpy as np
import rioxarray as rxr
from joblib import Parallel, delayed
from pathlib import Path
from typing import Union, Dict

## Set horizontal grid resolution and CRS

In [66]:
resolution = 450
crs = "EPSG:3413"

## Read IMBIE and Calfin using geopandas

In [105]:
imbie = gp.read_file("../data/imbie/GRE_Basins_IMBIE2_v1.3_w_shelves.gpkg").to_crs(crs)
calfin = gp.read_file("../data/calfin/termini_1972-2019_Greenland_closed_v1.0.shp").to_crs(crs)

In [92]:
geom_valid = calfin.geometry.make_valid()
calfin.geometry = geom_valid
calfin_dissolved = calfin.dissolve()

In [93]:
imbie_dissolved = imbie.dissolve()

In [137]:
date = pd.DatetimeIndex(calfin["Date"])
calfin["Date"] = date
calfin_ds = calfin.set_index(date)

In [127]:
#calfin_ds = calfin.set_index(date).groupby(by="Date")
nt = len(calfin)

In [129]:
for date, ds in calfin.groupby(by="Date"):
    
    print(date, len(ds))
    #create_ds(ds.dissolve(), imbie_union, date, geom)

1972-09-06 00:00:00 5
1972-09-07 00:00:00 4
1972-10-05 00:00:00 3
1972-10-14 00:00:00 2
1973-03-20 00:00:00 3
1973-03-21 00:00:00 3
1973-03-27 00:00:00 4
1973-03-29 00:00:00 2
1973-08-06 00:00:00 1
1974-04-10 00:00:00 3
1974-05-07 00:00:00 1
1975-03-23 00:00:00 6
1975-04-08 00:00:00 3
1975-04-09 00:00:00 1
1975-05-28 00:00:00 5
1975-06-11 00:00:00 4
1975-06-22 00:00:00 2
1975-07-19 00:00:00 4
1976-03-22 00:00:00 1
1976-03-31 00:00:00 1
1976-04-09 00:00:00 4
1976-04-18 00:00:00 1
1976-05-03 00:00:00 4
1976-05-04 00:00:00 2
1976-05-05 00:00:00 3
1976-05-29 00:00:00 1
1976-06-14 00:00:00 1
1978-07-28 00:00:00 4
1979-03-29 00:00:00 6
1979-04-08 00:00:00 5
1979-04-09 00:00:00 8
1979-07-14 00:00:00 3
1979-08-19 00:00:00 1
1980-09-02 00:00:00 7
1980-10-07 00:00:00 11
1980-10-10 00:00:00 5
1982-04-17 00:00:00 2
1982-09-12 00:00:00 4
1982-09-21 00:00:00 7
1982-09-22 00:00:00 4
1984-05-06 00:00:00 16
1984-05-07 00:00:00 6
1985-03-10 00:00:00 3
1985-03-23 00:00:00 4
1985-03-26 00:00:00 9
1985-03-

In [125]:
date

'1972-09-06'

In [117]:
ds.dissolve()

,geometry,GlacierID,Center_X,Center_Y,Latitude,Longitude,QualFlag,Satellite,Date,ImageID,GrnlndcN,OfficialN,AltName,RefName,Author
0,"MULTIPOLYGON (((327789.503 -2583059.536, 32777...",169,410262.354061,-2.556310e+06,66.424904,-35.882348,10,LM01,1972-09-06,LM01_L1TP_250013_19720906_20180429_01_T2,Kattilersorpia,Kattilersorpia,Glacier de France,Glacier de France,Cheng_D


In [118]:
imbie_union = imbie_dissolved.union(calfin_dissolved)

In [119]:
x_min = -653000
x_max = 879700
y_min = -632750
y_max = -3384350
bbox = [x_min, y_min, x_max, y_max]
geom = {
    "type": "Polygon",
    "crs": {"properties": {"name": "EPSG:3413"}},
    "bbox": bbox,
    "coordinates": [[
        (x_min, y_min), 
        (x_max, y_min), 
        (x_max, y_max), 
        (x_min, y_max),
        (x_min, y_min)  # Close the loop by repeating the first point
    ]]
}

In [132]:
def create_ds(ds1, ds2, date, geom: Dict, resolution: float = 450, crs: str = "EPSG:3413"):
    if len(ds1) > 0:
        ds = gp.GeoDataFrame(ds1, crs=crs)
        geom_valid = ds.geometry.make_valid()
        ds.geometry = geom_valid
        ds_dissolved = ds.dissolve()
        diff = ds2.difference(ds_dissolved.buffer(5))
        n = len(diff)
        diff_df = {"land_ice_area_fraction_retreat": np.ones(n)}
        diff_gp = gp.GeoDataFrame(data=diff_df, geometry=diff, crs=crs)
        ds = make_geocube(vector_data=diff_gp, geom=geom, resolution=(resolution, resolution))
        ds.land_ice_area_fraction_retreat["units"] = "1"
        ds = ds.expand_dims(time=[date])
        fn = Path(f"frontretreat_g{resolution}m_{date.year}-{date.month}-{date.day}.nc", encoding={"zlib": True, "complevel": 2})
        ds.to_netcdf(fn)
        del ds
        return fn

In [138]:
n_jobs = 8
result = Parallel(n_jobs=n_jobs)(
    delayed(create_ds)(ds,
        imbie_union, date, geom, resolution=resolution)
    for date, ds in tqdm(calfin_ds.groupby(by=pd.Grouper(freq="ME"))))

  0%|          | 0/562 [00:00<?, ?it/s]

/Users/andy/miniforge3/envs/pism-ragis/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


## Merge files and add time bounds

This could probably be done with *xarray* but setting the time axis and time_bounds correctly appears hard.

In [16]:
fn = Path(f"pism_g{resolution}m_frontretreat_calfin_1972_2019.nc")
!rm -rf $fn
!cdo -O -f nc4 -z zip_2 settbounds,1day -setattribute,land_ice_area_fraction_retreat@units="1" -setmisstoc,0 -mergetime frontretreat_g$gridm_*.nc $fn
!rm frontretreat_g$gridm_*

cdo(1) setmisstoc: Process started
cdo(2) mergetime: Process started
cdo    settbounds: Processed 6187640085 values from 1 variable over 297 timesteps [74.18s 951MB]


In [14]:
ls

GIS_mass_accounting.pdf
GRE_Basins_IMBIE2_v1.3.cpg
GRE_Basins_IMBIE2_v1.3.dbf
GRE_Basins_IMBIE2_v1.3.prj
GRE_Basins_IMBIE2_v1.3.qml
GRE_Basins_IMBIE2_v1.3.qpj
GRE_Basins_IMBIE2_v1.3.shp
GRE_Basins_IMBIE2_v1.3.shx
GRE_G0240_1985_2018_IDW_EXP_1.nc
GRE_G1800_1985_2018_IDW_EXP_1.nc
GRE_G900_1985_2018_IDW_EXP_1.nc
Ocean_Forcing.ipynb
Untitled.ipynb
analysis.ipynb
create_retreat_mask.ipynb
foo.nc
idw_merge_its_live.ipynb
jak_idw.nc
pism_g450m_frontretreat_calfin_1972_2019.nc.aux.xml


In [25]:
calfin_ds

,GlacierID,Center_X,Center_Y,Latitude,Longitude,QualFlag,Satellite,Date,ImageID,GrnlndcN,OfficialN,AltName,RefName,Author,geometry
Date,,,,,,,,,,,,,,,
1973-03-27,239,-316876.406283,-1.826173e+06,73.010730,-54.843912,10,LM01,1973-03-27,LM01_L1TP_021008_19730327_20180427_01_T2,Akullikassaap Sermia,Akullikassaap Sermia,None,Akullikassaap Sermia,Cheng_D,"POLYGON ((-315274.979 -1826407.729, -315275.44..."
1975-03-23,239,-316887.564103,-1.826131e+06,73.011086,-54.844474,10,LM02,1975-03-23,LM02_L1TP_018008_19750323_20180425_01_T2,Akullikassaap Sermia,Akullikassaap Sermia,None,Akullikassaap Sermia,Cheng_D,"POLYGON ((-315274.979 -1826407.729, -315275.44..."
1979-04-08,239,-316894.188037,-1.826121e+06,73.011163,-54.844727,10,LM02,1979-04-08,LM02_L1TP_019008_19790408_20180419_01_T2,Akullikassaap Sermia,Akullikassaap Sermia,None,Akullikassaap Sermia,Cheng_D,"POLYGON ((-315274.979 -1826407.729, -315275.44..."
1979-04-09,239,-316911.739426,-1.826125e+06,73.011098,-54.845240,10,LM02,1979-04-09,LM02_L1TP_020008_19790409_20180419_01_T2,Akullikassaap Sermia,Akullikassaap Sermia,None,Akullikassaap Sermia,Cheng_D,"POLYGON ((-315274.979 -1826407.729, -315275.44..."
1984-05-07,239,-316872.097806,-1.826114e+06,73.011265,-54.844095,10,LT05,1984-05-07,LT05_L1TP_015009_19840507_20170220_01_T1,Akullikassaap Sermia,Akullikassaap Sermia,None,Akullikassaap Sermia,Cheng_D,"POLYGON ((-315274.979 -1826407.729, -315275.44..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-10-12,20,-304472.838761,-1.851294e+06,72.805596,-54.339534,10,LC08,2018-10-12,LC08_L1TP_015009_20181012_20181030_01_T1,Sermeq,Sermeq,Upernavik Isstrøm SS,Upernavik Isstrom S,Cheng_D,"POLYGON ((-303751.095 -1855413.595, -303759.98..."
2019-02-26,20,-304450.994580,-1.851424e+06,72.804473,-54.338234,10,LC08,2019-02-26,LC08_L1TP_014009_20190226_20190309_01_T1,Sermeq,Sermeq,Upernavik Isstrøm SS,Upernavik Isstrom S,Cheng_D,"POLYGON ((-303486.576 -1855207.083, -303493.98..."
2019-03-21,20,-304355.734308,-1.851670e+06,72.802420,-54.334145,10,LC08,2019-03-21,LC08_L1TP_015009_20190321_20190403_01_T1,Sermeq,Sermeq,Upernavik Isstrøm SS,Upernavik Isstrom S,Cheng_D,"POLYGON ((-302542.368 -1855309.845, -302586.17..."


In [78]:
tqdm?

Init signature: tqdm(*args, **kwargs)
Docstring:      Experimental IPython/Jupyter Notebook widget using tqdm!
Init docstring:
Supports the usual `tqdm.tqdm` parameters as well as those listed below.

Parameters
----------
display  : Whether to call `display(self.container)` immediately
    [default: True].
File:           ~/miniforge3/envs/pism-ragis/lib/python3.11/site-packages/tqdm/auto.py
Type:           type
Subclasses:     

In [80]:
tqdm.tqdm?

Object `tqdm.tqdm` not found.


In [84]:
calfin_ds.groupby

Signature:
calfin_ds.groupby(
    by=None,
    axis: 'Axis | lib.NoDefault' = <no_default>,
    level: 'IndexLabel | None' = None,
    as_index: 'bool' = True,
    sort: 'bool' = True,
    group_keys: 'bool' = True,
    observed: 'bool | lib.NoDefault' = <no_default>,
    dropna: 'bool' = True,
) -> 'DataFrameGroupBy'
Docstring:
Group DataFrame using a mapper or by a Series of columns.

A groupby operation involves some combination of splitting the
object, applying a function, and combining the results. This can be
used to group large amounts of data and compute operations on these
groups.

Parameters
----------
by : mapping, function, label, pd.Grouper or list of such
    Used to determine the groups for the groupby.
    If ``by`` is a function, it's called on each value of the object's
    index. If a dict or Series is passed, the Series or dict VALUES
    will be used to determine the groups (the Series' values are first
    aligned; see ``.align()`` method). If a list or ndarray of